In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy as sp
%load_ext autoreload
%autoreload 2

# data path, constants, etc.
import sys
datapath = '/Users/nhamilto/Documents/ATom/coderepo/ATom/'
sys.path.append(datapath)

# plot things
import matplotlib.pyplot as plt
plt.style.use('seaborn-deep')

# Acoustic tomography functions
import atom_functions as ATF
import atom_classes as ATC

In [2]:
dset = ATC.dataset('/Users/nhamilto/Documents/ATom/data/20180913_Data_collection/')
dset.load_data_sample(0)

In [4]:
ATom_signals, travel_times, travel_time_inds = dset.extract_travel_times()

In [5]:
travel_times

,,frame 0,frame 1,frame 2,frame 3,frame 4,frame 5,frame 6,frame 7,frame 8,frame 9,...,frame 110,frame 111,frame 112,frame 113,frame 114,frame 115,frame 116,frame 117,frame 118,frame 119
speaker,mic,,,,,,,,,,,,,,,,,,,,,
S0,M0,2.790,2.795,2.790,2.790,2.790,2.790,2.795,2.795,2.805,2.800,...,2.790,2.790,2.790,2.790,2.795,2.790,2.795,2.795,2.795,2.795
S1,M0,89.760,89.765,89.750,89.700,89.685,89.680,89.655,89.600,89.630,89.665,...,89.945,89.905,89.950,89.955,89.965,89.955,89.985,89.945,89.950,89.945
S2,M0,129.530,129.510,129.450,129.555,129.475,129.440,129.385,129.375,129.315,129.300,...,129.835,129.775,129.765,129.765,129.720,129.670,129.685,129.675,129.655,129.635
S3,M0,224.905,224.975,225.040,225.070,225.050,225.000,225.015,224.975,224.855,224.770,...,224.685,225.515,225.490,224.625,225.430,225.410,225.385,225.400,225.365,225.360
S4,M0,232.940,232.945,232.985,232.905,232.830,232.900,232.935,232.885,232.970,232.950,...,232.600,233.400,233.340,233.300,233.310,233.290,233.250,233.280,233.265,233.280
S5,M0,237.215,236.335,237.050,236.210,237.085,236.250,237.120,237.270,237.340,237.360,...,237.435,237.400,237.345,236.520,237.340,236.495,237.245,237.170,236.280,237.100
S6,M0,199.160,199.185,199.220,199.275,199.290,199.320,199.305,199.330,199.315,199.315,...,199.300,199.295,199.335,200.155,199.335,199.340,199.365,199.395,199.390,199.360
S7,M0,123.050,123.035,123.015,122.960,122.925,123.005,123.050,123.070,123.065,123.035,...,122.850,122.940,123.005,123.010,122.990,122.960,122.965,122.980,122.960,122.920
S0,M1,94.505,95.300,94.500,95.300,95.350,94.600,94.645,94.720,94.745,94.690,...,95.195,94.450,95.190,94.390,94.385,95.160,94.370,94.400,94.405,95.150
